In [1]:
import csv
import cv2
import numpy as np
from PIL import Image 

In [2]:
lines=[]
data_dir="../data_given/data/"
#data_dir="../simulation_data/"
images_dir=data_dir+"IMG/"
csv_dir=data_dir+"driving_log.csv"
with open(csv_dir) as csvfile:
    reader=csv.reader(csvfile)
    for line in reader:
        lines.append(line)

In [3]:
def readimage(source_path,images_dir):
    filename=source_path.split("/")[-1]
    image_path=images_dir+filename
    image=cv2.imread(image_path)
    return image


In [4]:
images=[]
measurments=[]
lines.pop(0)

for line in lines:
    """
    source_path=line[0]
    image_center=readimage(source_path,images_dir)
    source_path=line[1]
    image_left=readimage(source_path,images_dir)
    source_path=line[2]
    image_right=readimage(source_path,images_dir)
    """
    center_img_dir = data_dir+'IMG/'+line[0].split('/')[-1]      
    center_image = cv2.imread(center_img_dir)         

    steering_center=float(line[3])
    correction = 0.2 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction

    #measurments.extend([steering_center,steering_left,steering_right])
    #images.extend([image_center,image_left,image_right])
    images.append(center_image)
    #images.append(image_left)
    #images.append(image_right)

    measurments.append(steering_center)
    #measurments.append(steering_left)
    #measurments.append(steering_right)
"""    
augmented_images,augmented_measurements=[],[]
for image,measurment in zip(images,measurments):
    augmented_images.append(image)
    augmented_measurements.append(measurment)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurment*-1.0)

image=[]
measurments=[]    
X_train=np.array(augmented_images)
Y_train=np.array(augmented_measurements)
"""
X_train=np.array(images)
Y_train=np.array(measurments)

In [5]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [6]:
import resource, sys
resource.setrlimit(resource.RLIMIT_STACK, (2**29,-1))
sys.setrecursionlimit(10**6)

import os
import csv
import cv2
import numpy as np
import sklearn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Cropping2D,MaxPooling2D,Dense, Activation, Flatten, Dropout,Lambda
from math import ceil
from random import shuffle
import tensorflow
print(tensorflow.keras.__version__)

model=Sequential()
model.add(Lambda(lambda x:x/255.0-.5,input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20), (0,0))))
model.add(Conv2D(24, (5, 2),activation="relu"))
model.add(Conv2D(36, (5, 2),activation="relu"))
model.add(Conv2D(48, (5, 2),activation="relu"))
model.add(Conv2D(64, 3, 3,activation="relu" ))
model.add(Conv2D(64, 3, 3,activation="relu" ))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss="mse",optimizer="adam")
model.fit(X_train,Y_train,validation_split=.2,shuffle=True,epochs=7,batch_size=64)
model.save("model.h5")


2.3.0-tf
Epoch 1/7
101/101 [==============================] - 36s 356ms/step - loss: 0.0382 - val_loss: 0.0114
Epoch 2/7
101/101 [==============================] - 19s 191ms/step - loss: 0.0102 - val_loss: 0.0108
Epoch 3/7
101/101 [==============================] - 19s 191ms/step - loss: 0.0099 - val_loss: 0.0110
Epoch 4/7
101/101 [==============================] - 20s 193ms/step - loss: 0.0094 - val_loss: 0.0104
Epoch 5/7
101/101 [==============================] - 19s 193ms/step - loss: 0.0093 - val_loss: 0.0102
Epoch 6/7
101/101 [==============================] - 19s 193ms/step - loss: 0.0088 - val_loss: 0.0105
Epoch 7/7
101/101 [==============================] - 18s 182ms/step - loss: 0.0085 - val_loss: 0.0104
